In [1]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tk


In [3]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt

def perform_object_detection(image_path, output_directory):
    # Load YOLO model and configuration
    net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')
    classes = []
    with open('coco.names', 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    # Load image
    image = cv2.imread(image_path)

    # Decrease the frame size (optional)
    frame_width = 800
    frame_height = 600
    image = cv2.resize(image, (frame_width, frame_height))

    # Get image dimensions
    height, width, channels = image.shape

    # Detect objects in the image
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Process the detected objects
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Change the confidence threshold as needed
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Find unique class IDs
    unique_class_ids = list(set(class_ids))

    # Assign unique colors to each class
    class_colors_unique = np.random.uniform(0, 255, size=(len(unique_class_ids), 3))

    # Non-maximum suppression to get rid of overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw the boxes and labels on the image with different colors
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            # Get the color corresponding to the class ID
            color = class_colors_unique[unique_class_ids.index(class_ids[i])].tolist()
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image, f"{label} {confidence:.2f}", (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 1, color, 2)

    # Save the result to the output directory
    output_image_path = f"{output_directory}/output_image.jpg"
    cv2.imwrite(output_image_path, cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    messagebox.showinfo("Object Detection", f"Object detection completed. Result saved to:\n{output_image_path}")

    # Display the result using matplotlib in GUI
    display_image(output_image_path)

def browse_image():
    image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png")])
    if image_path:
        output_directory = browse_output_directory()
        if output_directory:
            perform_object_detection(image_path, output_directory)

def browse_output_directory():
    selected_directory = filedialog.askdirectory()
    return selected_directory

def display_image(image_path):
    # Display the result using matplotlib in GUI
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
    ax.axis('off')  # Remove axis ticks and labels

    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.get_tk_widget().pack()

# Create the GUI window
root = tk.Tk()
root.title("Object Detection using YOLO")

# Add widgets
tk.Label(root, text="Object Detection using YOLO", font=("Helvetica", 16, "bold")).pack(pady=10)

# Browse buttons
tk.Button(root, text="Browse Image", command=browse_image).pack(pady=10)

# Run the GUI event loop
root.mainloop()
